In [2]:
import pandas as pd
import requests
from ipywidgets import interact, widgets

url = "https://partners.api.skyscanner.net/apiservices/v3/flights/indicative/search"
headers = {"x-api-key": "prtl6749387986743898559646983194"}

def get_flight_data(date, origin, destination):
    payload = {
        "query": {
            "market": "UK",
            "locale": "en-GB",
            "currency": "EUR",
            "queryLegs": [
                {
                    "originPlace": {"queryPlace": {"iata": origin}},
                    "destinationPlace": {"queryPlace": {"iata": destination}},
                    "fixedDate": date,
                }
            ],
        }
    }

    response = requests.post(url, json=payload, headers=headers)
    flight_data = response.json()

    flattened_results = []

    for quote_key, quote_data in flight_data["content"]["results"]["quotes"].items():
        price = quote_data["minPrice"]["amount"]
        origin_id = quote_data["outboundLeg"]["originPlaceId"]
        dest_id = quote_data["outboundLeg"]["destinationPlaceId"]
        airline_id = quote_data["outboundLeg"]["marketingCarrierId"]
        origin_name = flight_data["content"]["results"]["places"][origin_id]["name"]
        dest_name = flight_data["content"]["results"]["places"][dest_id]["name"]
        airline_name = flight_data["content"]["results"]["carriers"][airline_id]["name"]
        flight_dict = {
            "date": f"{date['year']}-{date['month']}-{date['day']}",
            "price": price,
            "origin": origin_name,
            "destination": dest_name,
            "company": airline_name,
        }
        flattened_results.append(flight_dict)

    return pd.DataFrame.from_records(flattened_results)

def search_flights():
    origin_input = widgets.Text(
        value='',
        placeholder='Enter origin airport',
        description='Origin:',
        disabled=False
    )
    dest_input = widgets.Text(
        value='',
        placeholder='Enter destination airport',
        description='Destination:',
        disabled=False
    )
    date_input = widgets.DatePicker(
        description='Date',
        disabled=False
    )
    display(origin_input)
    display(dest_input)
    display(date_input)
    
    def handle_search(sender):
        origin = origin_input.value.strip().upper()
        destination = dest_input.value.strip().upper()
        date = {"year": date_input.value.year, "month": date_input.value.month, "day": date_input.value.day}
        data = get_flight_data(date, origin, destination)
        output.clear_output()
        with output:
            display(data)
    
    def handle_reset(sender):
        origin_input.value = ''
        dest_input.value = ''
        date_input.value = None
        output.clear_output()
        
    search_button = widgets.Button(description='Search')
    reset_button = widgets.Button(description='Reset')
    display(widgets.HBox([search_button, reset_button]))
    output = widgets.Output()
    display(output)
    
    search_button.on_click(handle_search)
    reset_button.on_click(handle_reset)

In [3]:
search_flights()

Text(value='', description='Origin:', placeholder='Enter origin airport')

Text(value='', description='Destination:', placeholder='Enter destination airport')

DatePicker(value=None, description='Date')

Output()